Importando Librerias

In [1]:
!pip install Unidecode

In [2]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import numpy as np
import re
import warnings  
import unidecode

warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning,module='gensim') 

In [4]:
# Montamos el Google Drive para acceder facilmente a los archivos que necesitamos
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Cargando Dataset de tweets

In [5]:
Tweets = pd.read_pickle('/content/gdrive/My Drive/Dataset/Proyecto final DH/Data_tweet/PKL/tweets_cleaned.pkl')

In [6]:
Tweets = Tweets[(Tweets.lang=='es')&(Tweets.favorite_count>1)]

In [7]:
Tweets.head(2)
print(Tweets.shape)

(1118383, 23)


Caragando Dataset de Partidos politicos por usuarios

In [8]:
Usuarios_partidos=pd.read_csv('/content/gdrive/My Drive/Dataset/Proyecto final DH/Data_tweet/PKL/Copia de User_Partido.csv')

In [9]:
Usuarios_partidos.isnull().sum()

u.user_id                0
u.user_name              0
u.partido                0
u.partido_original    3685
dtype: int64

In [10]:
Usuarios_partidos=Usuarios_partidos[['u.user_id','u.partido']]

In [11]:
Usuarios_partidos=Usuarios_partidos.rename(columns={'u.user_id':'user_id','u.partido':'Partido'})
print(Usuarios_partidos.head(2))

               user_id          Partido
0            100014747  FRENTE DE TODOS
1  1000385229641670656  FRENTE DE TODOS


Cargando Dataset de usuarios de Tweeter

In [12]:
Usuarios=pd.read_csv('/content/gdrive/My Drive/Dataset/Proyecto final DH/ users_202008032307.csv')

In [13]:
df_merged = pd.merge(Tweets, Usuarios, on='user_id', how='outer')

In [14]:
df_merged.shape

(1152436, 43)

In [15]:
df=pd.merge(df_merged, Usuarios_partidos, on='user_id', how='inner')

In [16]:
df.shape

(597778, 44)

In [17]:
for i in df.columns:
    print("La columna",i, "tiene",round((df[i].isnull().sum()*100)/len(df[i]),3),"% de valores nulos")

La columna tweet_id tiene 0.005 % de valores nulos
La columna conversation_id tiene 0.005 % de valores nulos
La columna user_id tiene 0.0 % de valores nulos
La columna created_at_x tiene 0.005 % de valores nulos
La columna lang tiene 0.005 % de valores nulos
La columna retweet_count tiene 0.005 % de valores nulos
La columna favorite_count tiene 0.005 % de valores nulos
La columna reply_count tiene 0.005 % de valores nulos
La columna quote_count tiene 0.005 % de valores nulos
La columna is_reply tiene 0.005 % de valores nulos
La columna replied_id tiene 84.843 % de valores nulos
La columna replied_user_id tiene 84.843 % de valores nulos
La columna is_quote tiene 0.005 % de valores nulos
La columna quoted_id tiene 89.149 % de valores nulos
La columna full_text tiene 0.005 % de valores nulos
La columna source tiene 0.005 % de valores nulos
La columna scrapped_at_x tiene 0.005 % de valores nulos
La columna user_mentions_count tiene 0.005 % de valores nulos
La columna hashtags_count tiene 0

In [18]:
max_null=[]
for i in df.columns:
  if (df[i].isnull().sum()*100)/len(df[i]) >20:
    max_null.append(i)
print(max_null)

['replied_id', 'replied_user_id', 'quoted_id', 'replied_original', 'url', 'pinned_tweet']


In [19]:
df.drop(max_null,axis=1, inplace = True)

In [20]:
df.columns

Index(['tweet_id', 'conversation_id', 'user_id', 'created_at_x', 'lang',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'is_reply', 'is_quote', 'full_text', 'source', 'scrapped_at_x',
       'user_mentions_count', 'hashtags_count', 'urls_count', 'media_count_x',
       'full_text_cleaned', 'user_name', 'name', 'description', 'created_at_y',
       'geo_enabled', 'verified', 'profile_image_url',
       'profile_image_url_large', 'followers_count', 'friends_count',
       'listed_count', 'favourites_count', 'statuses_count', 'media_count_y',
       'tweets_scraped', 'tweets_scraped_at', 'scrapped_at_y', 'ignore',
       'Partido'],
      dtype='object')

In [21]:
df_final=df[['user_id','full_text_cleaned', 'user_name', 'name', 'description','Partido']]

In [22]:
df_final=df_final.dropna()

In [23]:
df_final.head(2)

,user_id,full_text_cleaned,user_name,name,description,Partido
0,69668594,🇦 🇷 patria construimos juntos # felizdíadelapa...,LupeTagliaferri,Guadalupe Tagliaferri,Vehemente pero no necia. Vivo sin odio. Senado...,JUNTOS POR EL CAMBIO
1,69668594,día prevención violencia noviazgo sigamos desn...,LupeTagliaferri,Guadalupe Tagliaferri,Vehemente pero no necia. Vivo sin odio. Senado...,JUNTOS POR EL CAMBIO


In [208]:
X=df_final[ 'description']
y=df_final['Partido']

In [209]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing import sequence
import keras
from keras.layers import Embedding, Dense, Flatten,BatchNormalization, Dropout
from keras.preprocessing.text import Tokenizer

In [210]:
x_train, x_test, y_train , y_test = train_test_split(X, y ,test_size=0.3, random_state=42)

In [211]:
print(x_test.shape)
print(x_train.shape)
print(y_test.shape)
print(y_train.shape)

(174681,)
(407587,)
(174681,)
(407587,)


In [212]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

In [213]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [214]:
labels = LabelEncoder()
y_train = labels.fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = labels.transform(y_test)
y_test = to_categorical(y_test)

In [215]:
x_train = sequence.pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=max_len, padding='post')

In [216]:
tokenize = lambda text: text.split(' ')

class Corpus():
    def __init__(self, df, size):
        self.df = df
        self.rowids = df.index.values
        self.len = size

    def __iter__(self):
        rowids = np.random.choice(self.rowids, self.len, replace=False)
        for doc_id in rowids:
            yield tokenize(self.df.loc[doc_id].full_text)

    def __len__(self):
        return self.len

In [197]:
vocab_corpus = df_final.full_text_cleaned.apply(tokenize).values

In [198]:
vocab_corpus

array([list(['🇦', '🇷', 'patria', 'construimos', 'juntos', '#', 'felizdíadelapatria', '#', '25demayo', 'URL']),
       list(['día', 'prevención', 'violencia', 'noviazgo', 'sigamos', 'desnaturalizando', 'relaciones', 'pareja', 'todas', 'aquellas', 'situaciones', 'violentas', 'no', 'demostraciones', 'amor', 'si', 'necesitás', 'ayuda', 'comunicate', 'manera', 'gratuita', '#', 'línea144', '24', 'horas', 'URL']),
       list(['saludamos', 'maestros', 'maestras', 'jardineras', 'día', 'especial', 'aquellos', 'desarrollan', 'tarea', 'centros', 'primera', 'infancia', 'chicos', 'ciudad', 'puedan', 'crecer', 'mejor', 'cuidado', 'igualdad', 'oportunidades', 'URL']),
       ...,
       list(['aún', 'viva', 'consume', 'pasión', '💋', '💗', 'griegos', 'no', 'escribían', 'necrológicas', 'sólo', 'hacían', 'pregunta', 'moría', 'hombre', 'pasión', '?']),
       list(['@', 'karakachoff1', '@', 'chechocipcic', '@', 'yaco_emiliano', '@', 'comuna1uno', '@', 'jrcapital', '@', 'libertad1065', '@', 'jrcomunauno1',

In [55]:
%load_ext cython
import time

start = time.time()
n_dim = 300
w2v = Word2Vec(vocab_corpus, workers=4, size=n_dim, min_count = 10, window = 10, sample = 1e-3, negative=10, sg=1, hs=0)
end = time.time()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [217]:
max_words = 250 ## numero maximo de palabras
max_len = 64 ## maximo numero de palabras x secuencia 
embedding_dim = 100 ## numero de dimensiones del embedding
n_clases = len(df_final['Partido'].unique())

In [218]:
df_final['Partido'].unique()

array(['JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS',
       'PTS - FRENTE DE IZQUIERDA', 'CONSENSO FEDERAL',
       'UNIDAD FEDERAL PARA EL DESARROLLO ',
       'FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES',
       'PARLAMENTARIO FEDERAL'], dtype=object)

# Modelo que estima ideologia politica segun la descripcion del perfil

Arquitectura 1

In [219]:
inputs = keras.Input(shape=(max_len,))
embedding = Embedding(max_words, embedding_dim, input_length = max_len)(inputs)
flat = Flatten()(embedding)
hidden = Dense(512, activation="relu")(flat)
hidden= Dropout(.6)(hidden)
hidden = BatchNormalization()(hidden)

hidden2 = Dense(512, activation="relu")(hidden)
hidden2= Dropout(.5)(hidden2)
hidden2 = BatchNormalization()(hidden2)

hidden3 = Dense(256, activation="relu")(hidden2)
hidden3= Dropout(.4)(hidden3)
hidden3 = BatchNormalization()(hidden3)

hidden4 = Dense(256, activation="relu")(hidden3)
hidden4= Dropout(.3)(hidden4)
hidden4 = BatchNormalization()(hidden4)

hidden5 = Dense(64, activation="relu")(hidden4)
hidden5= Dropout(.4)(hidden5)
hidden5 = BatchNormalization()(hidden5)

hidden6 = Dense(32, activation="relu")(hidden5)
hidden6= Dropout(.2)(hidden4)
hidden6 = BatchNormalization()(hidden6)

hidden7 = Dense(16, activation="relu")(hidden6)
hidden7= Dropout(.2)(hidden7)
hidden7 = BatchNormalization()(hidden7)

out = Dense(n_clases, activation="softmax")(hidden7)
model = keras.Model(inputs=inputs, outputs=out)

In [220]:
model.summary()

Model: "functional_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 64)]              0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 64, 100)           25000     
_________________________________________________________________
flatten_20 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_104 (Dense)            (None, 512)               3277312   
_________________________________________________________________
dropout_68 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_77 (Batc (None, 512)               2048      
_________________________________________________________________
dense_105 (Dense)            (None, 512)             

In [221]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.15, patience=3, restore_best_weights=True, verbose=1)
callbacks_list = [early_stopping]

"from keras.callbacks import EarlyStopping\n\nearly_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.15, patience=3, restore_best_weights=True, verbose=1)\ncallbacks_list = [early_stopping]"

In [241]:
"""
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(.005),
    metrics=["accuracy"],
)
"""


#history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

"""history_stop = model.fit(x_train, y_train,
                         epochs=20, batch_size=512,
                         callbacks=callbacks_list,
                         validation_split=0.2)"""



'history_stop = model.fit(x_train, y_train,\n                         epochs=20, batch_size=512,\n                         callbacks=callbacks_list,\n                         validation_split=0.2)'

Arquitectura 2

In [243]:
inputs = keras.Input(shape=(max_len,))
embedding = Embedding(max_words, embedding_dim, input_length = max_len)(inputs)
flat = Flatten()(embedding)
hidden = Dense(512, activation="relu")(flat)
hidden= Dropout(.3)(hidden)
hidden = BatchNormalization()(hidden)

hidden2 = Dense(256, activation="relu")(hidden)
hidden2= Dropout(.3)(hidden2)
hidden2 = BatchNormalization()(hidden2)

hidden3 = Dense(64, activation="relu")(hidden2)
hidden3= Dropout(.3)(hidden3)
hidden3 = BatchNormalization()(hidden3)

hidden4 = Dense(32, activation="relu")(hidden3)
hidden4= Dropout(.3)(hidden4)
hidden4 = BatchNormalization()(hidden4)



out = Dense(n_clases, activation="softmax")(hidden4)
model2 = keras.Model(inputs=inputs, outputs=out)

In [244]:
model2.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(.005),
    metrics=["accuracy"],
)

history_stop_2 = model2.fit(x_train, y_train,
                         epochs=5, batch_size=512,
                         callbacks=callbacks_list,
                         validation_split=0.2)

Epoch 1/5
637/637 [==============================] - 170s 266ms/step - loss: 1.2804 - accuracy: 0.8871 - val_loss: 1.2384 - val_accuracy: 0.9273
Epoch 2/5
637/637 [==============================] - 160s 251ms/step - loss: 1.2392 - accuracy: 0.9262 - val_loss: 1.2909 - val_accuracy: 0.8742
Epoch 3/5
637/637 [==============================] - 172s 270ms/step - loss: 1.2367 - accuracy: 0.9287 - val_loss: 1.2327 - val_accuracy: 0.9333
Epoch 4/5
637/637 [==============================] - 165s 260ms/step - loss: 1.2326 - accuracy: 0.9328 - val_loss: 1.2897 - val_accuracy: 0.8761
Epoch 00004: early stopping


In [245]:
test_scores2 = model2.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores2[0])
print("Test accuracy:", test_scores2[1])

5459/5459 - 41s - loss: 1.2411 - accuracy: 0.9245
Test loss: 1.2411054372787476
Test accuracy: 0.9245424270629883


Arquitetura 3

In [238]:
inputs = keras.Input(shape=(max_len,))
embedding = Embedding(max_words, embedding_dim, input_length = max_len)(inputs)
flat = Flatten()(embedding)
hidden = Dense(64, activation="relu")(flat)
hidden2 = Dense(16, activation="relu")(hidden)
out = Dense(n_clases, activation="softmax")(hidden2)
model = keras.Model(inputs=inputs, outputs=out)

In [275]:
model.summary()

Model: "functional_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 64)]              0         
_________________________________________________________________
embedding_22 (Embedding)     (None, 64, 100)           25000     
_________________________________________________________________
flatten_22 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_117 (Dense)            (None, 64)                409664    
_________________________________________________________________
dense_118 (Dense)            (None, 16)                1040      
_________________________________________________________________
dense_119 (Dense)            (None, 7)                 119       
Total params: 435,823
Trainable params: 435,823
Non-trainable params: 0
_______________________________________________

In [239]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(.005),
    metrics=["accuracy"],
)

history_stop = model.fit(x_train, y_train,
                         epochs=5, batch_size=512,
                         callbacks=callbacks_list,
                         validation_split=0.2)

Epoch 1/5
637/637 [==============================] - 30s 47ms/step - loss: 1.2889 - accuracy: 0.8766 - val_loss: 1.2311 - val_accuracy: 0.9341
Epoch 2/5
637/637 [==============================] - 29s 45ms/step - loss: 1.2248 - accuracy: 0.9406 - val_loss: 1.2212 - val_accuracy: 0.9436
Epoch 3/5
637/637 [==============================] - 28s 45ms/step - loss: 1.2214 - accuracy: 0.9440 - val_loss: 1.2176 - val_accuracy: 0.9478
Epoch 4/5
637/637 [==============================] - 29s 46ms/step - loss: 1.2190 - accuracy: 0.9464 - val_loss: 1.2161 - val_accuracy: 0.9493
Epoch 00004: early stopping


In [240]:
test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

5459/5459 - 9s - loss: 1.2327 - accuracy: 0.9326
Test loss: 1.2327240705490112
Test accuracy: 0.9325799345970154


In [246]:
#El mejor modelos es el 3 ya que obtuvo un buen accuracy y tardo menos tiempo en correr, ademas de ser una arquitectura muy simple

In [297]:
predictions_by_desc = model.predict(x_test)

In [298]:
np.argmax(predictions_by_desc[0])

2

#Modelo que estima el ideologia politica segun los tweet que escribe un usuario

In [277]:
X_2=df_final[ 'full_text_cleaned']
y_2=df_final['Partido']

In [278]:
x_train_2, x_test_2, y_train_2 , y_test_2 = train_test_split(X_2, y_2 ,test_size=0.3, random_state=42)

In [279]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train_2)

In [280]:
x_train_2 = tokenizer.texts_to_sequences(x_train_2)
x_test_2 = tokenizer.texts_to_sequences(x_test_2)

In [281]:
x_train_2 = sequence.pad_sequences(x_train_2, maxlen=max_len, padding='post')
x_test_2 = sequence.pad_sequences(x_test_2, maxlen=max_len, padding='post')

In [290]:
labels = LabelEncoder()
y_train_2 = labels.fit_transform(y_train_2)
y_train_2 = to_categorical(y_train_2)

y_test_2 = labels.transform(y_test_2)
y_test_2 = to_categorical(y_test_2)

In [291]:
max_words = 250 ## numero maximo de palabras
max_len = 64 ## maximo numero de palabras x secuencia 
embedding_dim = 100 ## numero de dimensiones del embedding
n_clases = len(df_final['Partido'].unique())

In [292]:
inputs_ = keras.Input(shape=(max_len,))
embedding_ = Embedding(max_words, embedding_dim, input_length = max_len)(inputs_)
flat_ = Flatten()(embedding_)
hidden_ = Dense(64, activation="relu")(flat_)
hidden2_ = Dense(16, activation="relu")(hidden_)
out_ = Dense(n_clases, activation="softmax")(hidden2_)
model_4 = keras.Model(inputs=inputs_, outputs=out_)

In [293]:
model_4.summary()

Model: "functional_54"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 64)]              0         
_________________________________________________________________
embedding_28 (Embedding)     (None, 64, 100)           25000     
_________________________________________________________________
flatten_28 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_137 (Dense)            (None, 64)                409664    
_________________________________________________________________
dense_138 (Dense)            (None, 16)                1040      
_________________________________________________________________
dense_139 (Dense)            (None, 7)                 119       
Total params: 435,823
Trainable params: 435,823
Non-trainable params: 0
_______________________________________________

In [295]:
model_4.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(.005),
    metrics=["accuracy"],
)

history_stop_4 = model.fit(x_train_2, y_train_2,
                         epochs=5, batch_size=512,
                         callbacks=callbacks_list,
                         validation_split=0.2)

Epoch 1/5
637/637 [==============================] - 31s 48ms/step - loss: 1.4860 - accuracy: 0.6794 - val_loss: 1.4863 - val_accuracy: 0.6791
Epoch 2/5
637/637 [==============================] - 31s 48ms/step - loss: 1.4856 - accuracy: 0.6798 - val_loss: 1.4863 - val_accuracy: 0.6791
Epoch 3/5
637/637 [==============================] - 33s 51ms/step - loss: 1.4856 - accuracy: 0.6798 - val_loss: 1.4863 - val_accuracy: 0.6791
Epoch 4/5
637/637 [==============================] - 32s 50ms/step - loss: 1.4856 - accuracy: 0.6798 - val_loss: 1.4863 - val_accuracy: 0.6791
Epoch 00004: early stopping


In [355]:
test_scores4 = model_4.evaluate(x_test_2, y_test_2, verbose=2)
print("Test loss:", test_scores4[0])
print("Test accuracy:", test_scores4[1])

5459/5459 - 8s - loss: 1.4843 - accuracy: 0.6812
Test loss: 1.484320044517517
Test accuracy: 0.6811788082122803
